In [ ]:
#|default_exp utils

In [ ]:
#|export
import wandb
from fastai.vision.all import *
from fastai.callback.wandb import *
import torchvision.transforms as T
from torchvision.datasets import MNIST, CIFAR10

### Basic dataset methods

In [ ]:
#|export
FLATTEN_TFM = T.Lambda(lambda x: x.view(-1,28*28).squeeze())
def get_mnist_dls(bs=256, num_workers=None, tfms=FLATTEN_TFM):
    mean, std = 0.130652368068695068, 0.307504087686538696
    tfms = tfms if isinstance(tfms,list) else [tfms]
    tfm = T.Compose([T.ToTensor(),T.Normalize((mean,), (std,)), *tfms])
    train = MNIST('../data', train=True, download=True, transform=tfm)
    test = MNIST('../data', train=False, download=True, transform=tfm)
    return DataLoaders(TfmdDL(train, bs, True, num_workers),TfmdDL(test, bs, False, num_workers))

In [ ]:
#|export
FLATTEN_CIFAR_TFM = [T.Lambda(lambda x: x.view(-1,32*32*3).squeeze())]
def get_cifar_dls(bs=64, num_workers=None, tfms=FLATTEN_CIFAR_TFM):
    mean, std = (0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616)
    tfms = T.Compose([T.ToTensor(),T.Normalize(mean, std), *tfms])
    train = CIFAR10('../data', train=True, download=True, transform=tfms)
    test = CIFAR10('../data', train=False, download=True, transform=tfms)
    return DataLoaders(TfmdDL(train, bs, True, num_workers),TfmdDL(test, bs, False, num_workers))

In [ ]:
#|export
def runs_sweep(sweep_cfg:dict, project=None, count=5, sweep_name=None, sweep_id=None):
    '''returns a decorator that runs a function in a sweep, first argument will be sweep_cfg initialized by wandb'''
    sweep_cfg['name'] = sweep_cfg.get('name', sweep_name)
    def _f(fn, *args, **kwargs):
        def run():
            with wandb.init() as run:
                fn(wandb.config, run, *args,**kwargs)
        nonlocal sweep_id
        if sweep_id is None: sweep_id = wandb.sweep(sweep_cfg, project=project)
        wandb.agent(sweep_id, run, count=count)
    return _f

### Configure callabacks 

In [ ]:
#|export
class TinyWandbCallback(WandbCallback): pass

@patch
def gather_args(self:Learner):
    "Gather config parameters accessible to the learner"
    args = {}
    # input dimensions
    try:
        n_inp = self.dls.train.n_inp
        xb = self.dls.valid.one_batch()[:n_inp]
        args.update({f'input dim {i+1}':d for n in range(n_inp) for i,d in enumerate(list(detuplify(xb[n]).shape))})
    except: print(f'Could not gather input dimensions')
    # other useful information
    with ignore_exceptions():
        args['batch per epoch'] = len(self.dls.train)
        args['model parameters'] = total_params(self.model)[0]
        args['device'] = self.dls.device.type
        args['frozen'] = bool(self.opt.frozen_idx)
        args['frozen idx'] = self.opt.frozen_idx
        args['dataset.tfms'] = f'{self.dls.dataset.tfms}'
        args['dls.after_item'] = f'{self.dls.after_item}'
        args['dls.before_batch'] = f'{self.dls.before_batch}'
        args['dls.after_batch'] = f'{self.dls.after_batch}'
    return args

Callback to extract expert distributions

In [ ]:
#|export
class ProbsDistrCB(Callback):
    '''Gets probability distribution of module on train set and logs to wandb if enabled'''
    def __init__(self, probs_attr='probs', wandb=False, module=None, sample_size=30):
        store_attr()
 
    def before_fit(self): 
        self.module = ifnone(self.module, self.learn.model)
        if self.wandb: self._wandb_step = wandb.run.step - 1
        self.all_blocks, self.all_ys = [], []

    def before_epoch(self):  
        self.blocks, self.ys = [], []
        if self.epoch == self.n_epoch-1: self.xs, self.preds = [], []
    
    def after_batch(self): 
        if not self.training: return
        if self.wandb:  self._wandb_step += 1
        if not hasattr(self, 'n_blocks'): self.n_blocks = getattr(self.module, self.probs_attr).squeeze().shape[-1]

        self.blocks.append(getattr(self.module, self.probs_attr).argmax(1).squeeze())
        self.ys.append(self.yb[0])
        if self.epoch == self.n_epoch-1:
            self.xs.append(self.xb[0])
            self.preds.append(self.pred.argmax(1))

    def after_epoch(self):
        self.all_blocks.append(torch.cat(self.blocks))
        self.all_ys.append(torch.cat(self.ys))
        if self.wandb: 
            fig, axs = subplots()
            self.show(-1, ax=axs[0], show=False)
            handles, labels = axs[0].get_legend_handles_labels()
            fig.legend(handles, labels)
            wandb.log({"Blocks data distribution": fig}, step = self._wandb_step)
        if self.epoch == self.n_epoch-1: 
            self.xs, self.preds = torch.cat(self.xs), torch.cat(self.preds)

    def after_fit(self):
        if not self.wandb: return
        idx = torch.randperm(len(self.xs))[:self.sample_size]
        bs, xs, ps, ys = self.all_blocks[-1][idx], self.xs[idx], self.preds[idx], self.all_ys[-1][idx]
        cols = ['block','image','pred','target']
        data = [[b, wandb.Image(x.view(28,28)), p, y] for b,x,p,y in zip(bs,xs,ps,ys)]
        wandb.log({"samples": wandb.Table(data=data, columns=cols)})
    
    def show(self, epoch_idx, ax=None, figsize=(8,6), show=True):
        blocks, ys = self.all_blocks[epoch_idx], self.all_ys[epoch_idx]
        ax = subplots(figsize=figsize)[1][0] if ax is None else ax
        bottom, bins = torch.zeros(self.n_blocks), L(range(self.n_blocks)).map(str)
        for d in range(10):
            hist = to_cpu(torch.bincount(blocks[ys==d], minlength=self.n_blocks))
            ax.bar(bins, hist, label=str(d),bottom=bottom)
            bottom += hist
        if show: 
             ax.legend(); plt.show()


    def show_all_epochs(self, n_epoch, ncols=None):
        n_epoch = 10
        ncols = ifnone(ncols, int(n_epoch**0.5))
        fig, axes = subplots(ncols=ncols, nrows=math.ceil(n_epoch/ncols), figsize=(12,5), layout="constrained")
        for i in range(n_epoch):
            ax =  axes[i//ncols][i%ncols] if math.ceil(n_epoch/ncols)>1 else axes[i%ncols]
            ax.set_yticklabels([])
            self.show(i, ax=ax, show=False)
        handles, labels = ax.get_legend_handles_labels()
        fig.legend(handles, labels)

In [ ]:
#|export
class GetGradCB(Callback):
    '''Get grads from modules'''
    def __init__(self, modules: list): self.modules = modules
    def before_fit(self): 
        self.grads = {m:[] for m in self.modules}

    def after_fit(self): self.grads = {k:torch.stack(v) for k, v in self.grads.items()}
    
    def after_backward(self):
        for mod in self.modules:
            grad = getattr(mod, 'weight',mod)
            grad = getattr(grad, 'grad', None)
            if grad is None: continue
            self.grads[mod].append(torch.clone(grad.detach()))

    def show(self, mod, normalize=lambda x: x.abs().mean(tuple(range(1, x.ndim))), ax=None, figsize=(8,6), beta=0.98, **kwargs):
        ax = subplots(figsize=figsize)[1][0] if ax is None else ax
        grads = smooth_avg(normalize(self.grads[mod])).cpu().numpy()
        if grads.ndim==1: grads = grads[:,None]
        for i in range(grads.shape[-1]): ax.plot(grads[:,i], label=ifnone(kwargs.pop('label',None), i), **kwargs)
        ax.legend()
        

#|export
def smooth_avg(tensor, dim=0, beta=0.95):
    tensor = tensor.swapaxes(0, dim)
    res=torch.zeros_like(tensor, device=tensor.device)
    for i in range(tensor.shape[0]):
        val = torch.lerp(tensor[i], res[max(0,i-1)], beta)
        res[i] = val
    fr = 1-beta**torch.arange(1,res.shape[0]+1, device=res.device)
    return (res.swapaxes(0,-1)/fr).swapaxes(0, -1).swapaxes(0, dim)